In [1]:
import numpy as np     #只需要下载numpy库即可
import random
import GridWorld_v2

In [2]:
gamma = 0.9   #折扣因子，越接近0越近视

rows = 5      #记得行数和列数这里要同步改
columns = 5

# gridworld = GridWorld_v2.GridWorld_v2(rows=rows, columns=columns, forbiddenAreaNums=8, targetNums=2, seed = 52,forbiddenAreaScore=-10)
# gridworld = GridWorld_v2.GridWorld_v2(desc = [".#",".T"])             #赵老师4-1的例子
# gridworld = GridWorld_v2.GridWorld_v2(desc = ["##.T","...#","...."])  #随便弄的例子
gridworld = GridWorld_v2.GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = [".....",".##..","..#..",".#T#.",".#..."]) 
#gridworld = GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = ["T."]) 
gridworld.show()


value = np.zeros(rows*columns)       #初始化可以任意，也可以全0
qtable = np.zeros((rows*columns,5))  #初始化，这里主要是初始化维数，里面的内容会被覆盖所以无所谓

# np.random.seed(50)
policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
gridworld.showPolicy(policy)

⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️
➡️⬇️➡️⬇️🔄
⬆️🔄⏬⬇️🔄
⬆️➡️⏩️⬇️⬆️
➡️🔄✅🔄➡️
➡️⏪➡️⬅️⬆️


In [12]:
epsilon = 0.99
p1 = 1-epsilon * (4/5)
p2 = epsilon/5
d = {1:p1, 0:p2}
a2 = np.vectorize(d.get)(policy)

In [13]:
from IPython.display import clear_output
clear_output(wait=True)

In [14]:
#通过采样的方法计算action value，model free的话意味着不知道整个gridworld的概率了，所以不能直接套贝尔曼方程迭代求解
policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
gridworld.show()
gridworld.showPolicy(policy)
print("random policy")


trajectorySteps = 20000
learning_rate = 0.01
epsilon = 0.5
# qtable = np.zeros((rows*columns,5))    #生成Qtable，也就是action-value-table
state_value = np.zeros((rows * columns))

num_episodes = 6000
for episode in range(num_episodes):
    # 
    if(epsilon > 0.001) :
        epsilon -= 0.001
    else:
        epsilon = 0.001
        
    p1 = 1-epsilon * (4/5)
    p0 = epsilon/5
    # trajectorySteps = int(20+epsilon*1000)
    print("trajectorySteps",trajectorySteps)
    print(f"epision:{epsilon}, p1:{p1}, p0:{p0}")
    
    d = {1:p1, 0:p0}
    policy_epsilon = np.vectorize(d.get)(policy)
    
    i = random.randint(0,24)  #初始状态
    j = random.randint(0,4)
    
    cnt = [0 for i in range(25)]
    
    Trajectory = gridworld.getTrajectoryScore(nowState=i, action=j, policy=policy_epsilon, steps=trajectorySteps, stop_when_reach_target=True)
    if(len(Trajectory)>5000):
        continue
    clear_output(wait=True)
    # 注意这里的返回值是大小为(trajectorySteps+1)的元组列表，因为把第一个动作也加入进去了
    steps = len(Trajectory) - 1
    for k in range(steps,-1,-1):
        tmpstate, tmpaction, tmpscore, nextState, nextAction  = Trajectory[k]
        cnt[tmpstate] += 1
        TD_error = state_value[tmpstate] - (tmpscore + gamma * state_value[nextState])
        state_value[nextState] = state_value[tmpstate] - learning_rate * TD_error
    print(np.array(cnt).reshape(5,5))
    for i in range(rows * columns):
        for j in range(5):
            score,nextState = gridworld.getScore(i,j) 
            qtable[i][j] = score + gamma * value[nextState]
    
    print(state_value.reshape(5,5))
    gridworld.showPolicy(policy)
    print(np.array(state_value).mean())
    
    
    policy = np.eye(5)[np.argmax(qtable,axis=1)]  #qtable的最优值作为更新策略，并用独热码来表示
    policy_epsilon = np.vectorize(d.get)(policy)
        
    # print(np.array(cnt).reshape(5,5))


    


    

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 1 0]
 [0 0 1 0 0]
 [0 0 0 0 0]]
[[-0.50927475 -0.14070731 -0.14324995 -0.38438712 -0.1530971 ]
 [-0.03856409 -0.53091662 -0.21396443 -0.10593053 -0.18790431]
 [-0.03852397 -0.14134947 -0.19588346 -0.0940211  -0.09507617]
 [-0.04846244 -0.59073003 -0.30240289 -0.28572883 -0.16706666]
 [-0.04855244 -0.49212653 -0.49516866 -0.15633104 -0.14907693]]
➡️➡️➡️➡️⬇️
⬆️⏫️⏫️⬆️⬆️
⬆️⬅️⏬⬆️⬆️
⬆️⏩️✅⏪⬆️
⬆️⏩️⬆️➡️⬆️
-0.22833987322135776
trajectorySteps 20000
epision:0.21799999999999975, p1:0.8256000000000002, p0:0.04359999999999995


KeyboardInterrupt: 